In [34]:
#importing libs
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
#loading dataFrames
LT_df = pd.read_csv('LT_db.csv', index_col=0)
MT_df = pd.read_csv('MT_db.csv', index_col=0)
ST_df = pd.read_csv('ST_db.csv', index_col=0)
LT_df.head()

,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,0.076982,0.073146,0.076875,0.080231,0.080052,0.074072,0.071745,0.073260,0.073215,0.074249,...,0.081482,0.078695,0.079444,0.079535,0.075231,0.074109,0.073722,0.070257,0.069479,0.071449
1,0.025159,0.028201,0.032614,0.029712,0.036319,0.050134,0.051584,0.057166,0.058675,0.052958,...,0.097371,0.106728,0.107960,0.109923,0.107925,0.113627,0.113551,0.114614,0.117321,0.121645
2,0.057198,0.061475,0.062880,0.062228,0.062439,0.059846,0.061742,0.061078,0.062800,0.064839,...,0.079657,0.078875,0.081551,0.086172,0.080837,0.091557,0.094971,0.100685,0.106954,0.111544
3,0.072266,0.078686,0.080705,0.079294,0.080649,0.084033,0.073463,0.074652,0.080304,0.080940,...,0.080507,0.081097,0.079449,0.071548,0.069084,0.066095,0.063184,0.060153,0.062863,0.064928
4,0.049170,0.050382,0.057310,0.051338,0.043277,0.046501,0.053381,0.068414,0.062216,0.067075,...,0.085228,0.090190,0.086500,0.087039,0.099709,0.104405,0.113078,0.111316,0.115982,0.093622


In [36]:
years = len(LT_df.columns)

In [37]:
#User Inputs
epochs = 50
cost_of_action = 50
size_of_action_space = 10

In [38]:
LT_values = LT_df.values[:-1].T
Population_LT = LT_df.values[-1]

In [39]:
LT_action_space = np.round(
    np.linspace(min(LT_values.ravel()), max(LT_values.ravel()),
                size_of_action_space + 2), 3)
LT_action_space = LT_action_space[1:-1]
print('LT_action_space: ', LT_action_space)

LT_action_space:  [0.035 0.057 0.079 0.101 0.122 0.144 0.166 0.188 0.209 0.231]


In [40]:
MT_values = MT_df.values[:-1].T
Population_MT = MT_df.values[-1]

In [41]:
MT_action_space = np.round(
    np.linspace(min(MT_values.ravel()), max(MT_values.ravel()),
                size_of_action_space + 2), 3)
MT_action_space = MT_action_space[1:-1]
print('MT_action_space: ', MT_action_space)

MT_action_space:  [0.015 0.022 0.029 0.036 0.044 0.051 0.058 0.066 0.073 0.08 ]


In [42]:
ST_values = ST_df.values[:-1].T
Population_ST = ST_df.values[-1]
Population_array = Population_LT + Population_MT + Population_ST

In [43]:
ST_action_space = np.round(
    np.linspace(min(ST_values.ravel()), max(ST_values.ravel()),
                size_of_action_space + 2), 3)
ST_action_space = ST_action_space[1:-1]
print('ST_action_space: ', ST_action_space)

ST_action_space:  [0.008 0.015 0.022 0.029 0.036 0.043 0.05  0.057 0.064 0.071]


In [44]:
#Q-tables
Q_LT = np.zeros((years + 1, size_of_action_space))
Q_MT = np.zeros((years + 1, size_of_action_space))
Q_ST = np.zeros((years + 1, size_of_action_space))

#Q-tables for every epoch that stores all max values from
Q_LT_per_epoch = []
Q_MT_per_epoch = []
Q_ST_per_epoch = []

In [45]:
global_state_per_epoch = []

In [46]:
#defining the weight factors of immediate rewards
LT_reward_factor = 0.4
MT_reward_factor = 0.5
ST_reward_factor = 0.6
cumulative_reward = 0  #initializing cumulative reward, which is 0 to start with

In [47]:
#creating a list to store the cumulative reward for each epoch
cumulative_reward_per_epoch = []

#creating a list to store the immediate rewards for each epoch
immediate_rewards_per_epoch = []

In [48]:
#epsilons are initialized within epoch.

#rate of decrement
LT_epsilon_decay = 0.95
MT_epsilon_decay = 0.9
ST_epsilon_decay = 0.85

#min value
LT_epsilon_min = 0.1
MT_epsilon_min = 0.06
ST_epsilon_min = 0.03

In [49]:
#alpha <=> Learning rate (will be initialized within epoch)
alpha_min = 0.01  #min alpha
alpha_decay = 0.995  #rate of decrement

gamma = 0.7  #<=>reward discount

<br>



<br>

In [50]:
#defining best actions from action space
Max_Q_LT = 0
Max_Q_MT = 0
Max_Q_ST = 0

for epoch in range(0, epochs):

    #initalize variables
    global_state_of_co2_emission = 4.025  #CO2 emmision in 1984

    cumulative_reward = 0  #intially 0

    alpha = 0.1  #learning rate
    LT_epsilon = 0.9
    MT_epsilon = 0.8
    ST_epsilon = 0.7

    for year in range(0, years):

        #LT
        if np.random.rand() <= LT_epsilon:
            LT_action = random.choice(LT_values[year])
        else:
            LT_action = LT_action_space[Max_Q_LT]

        LT_immediate_reward = -1 * LT_action * LT_reward_factor + cost_of_action * LT_action
        Q_LT[year, absw(LT_action_space - LT_action).argmin()] = round(
            (1 - alpha) * Q_LT[year, Max_Q_LT] + alpha *
            (LT_immediate_reward + gamma * np.amax(Q_LT[year + 1, :])), 3)
        
        #MT
        if np.random.rand() <= MT_epsilon:
            MT_action = random.choice(MT_values[year])
        else:
            MT_action = MT_action_space[Max_Q_MT]

        MT_immediate_reward = -1 * MT_action * MT_reward_factor + cost_of_action * MT_action
        Q_MT[year, abs(MT_action_space - MT_action).argmin()] = round(
            (1 - alpha) * Q_MT[year, Max_Q_MT] + alpha *
            (MT_immediate_reward + gamma * np.amax(Q_MT[year + 1, :])), 3)
        
        #ST
        if np.random.rand() <= ST_epsilon:
            ST_action = random.choice(ST_values[year])
        else:
            ST_action = ST_action_space[Max_Q_ST]

        ST_immediate_reward = -1 * ST_action * ST_reward_factor + cost_of_action * ST_action
        Q_ST[year, abs(ST_action_space - ST_action).argmin()] = round(
            (1 - alpha) * Q_ST[year, Max_Q_ST] + alpha *
            (ST_immediate_reward + gamma * np.amax(Q_ST[year + 1, :])), 3)
        
        #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
        cumulative_reward += LT_immediate_reward + MT_immediate_reward + ST_immediate_reward - cost_of_action*((LT_action_space[np.argmax(Q_LT[year, :])]) - (MT_action_space[np.argmax(Q_MT[year, :])]) - (ST_action_space[np.argmax(Q_ST[year, :])]))
        
        #decaying learning rate and agent's epsilon values
        alpha = alpha*alpha_decay if (alpha > alpha_min) else alpha
        LT_epsilon = LT_epsilon*LT_epsilon_decay if (LT_epsilon > LT_epsilon_min) else LT_epsilon
        MT_epsilon = MT_epsilon*MT_epsilon_decay if (MT_epsilon > MT_epsilon_min) else MT_epsilon
        ST_epsilon = ST_epsilon*ST_epsilon_decay if (ST_epsilon > ST_epsilon_min) else ST_epsilon
        
        global_state_of_co2_emission += ((LT_action*Population_LT[year]) + (MT_action*Population_MT[year]) + (ST_action*Population_ST[year]))/Population_array[year]
    
    
    Q_LT_per_epoch.append(np.argmax(Q_LT, axis=1).tolist())
    Q_MT_per_epoch.append(np.argmax(Q_MT, axis=1).tolist())
    Q_ST_per_epoch.append(np.argmax(Q_ST, axis=1).tolist())
  
    cumulative_reward_per_epoch.append(cumulative_reward)
    immediate_rewards_per_epoch.append([LT_immediate_reward, MT_immediate_reward, ST_immediate_reward])
    global_state_per_epoch.append(global_state_of_co2_emission)


In [51]:
immediate_rewards_per_epoch = np.array(immediate_rewards_per_epoch,copy=False)

In [52]:
#evaluating the trained model
print('\n')
print("Best Epoch based on Cumulative Reward: ",
      np.argmax(cumulative_reward_per_epoch))
print(
    f"Highest Cumulative Reward: {round(np.amax(cumulative_reward_per_epoch), 2)}"
)
print(f"Best Epoch based on Global State: {np.argmin(global_state_per_epoch)}")
print(f"Lowest Global State: {round(np.min(global_state_per_epoch), 2)}")
print('\n')
print(
    f"Best Epoch based on LT's Immediate Reward: {np.argmax(immediate_rewards_per_epoch[:, 0])}"
)
print(
    f"Highest Immediate Reward for LT: {round(np.max(immediate_rewards_per_epoch[:, 0]), 2)}"
)
print(
    f"Best Epoch based on MT's Immediate Reward: {np.argmax(immediate_rewards_per_epoch[:, 1])}"
)
print(
    f"Highest Immediate Reward for MT: {round(np.max(immediate_rewards_per_epoch[:, 1]), 2)}"
)
print(
    f"Best Epoch based on ST's Immediate Reward: {np.argmax(immediate_rewards_per_epoch[:, 2])}"
)
print(
    f"Highest Immediate Reward for ST: {round(np.max(immediate_rewards_per_epoch[:, 2]), 2)}"
)

print('\n')



Best Epoch based on Cumulative Reward:  18
Highest Cumulative Reward: 106.14
Best Epoch based on Global State: 6
Lowest Global State: 4.43


Best Epoch based on LT's Immediate Reward: 28
Highest Immediate Reward for LT: 12.54
Best Epoch based on MT's Immediate Reward: 0
Highest Immediate Reward for MT: 0.74
Best Epoch based on ST's Immediate Reward: 45
Highest Immediate Reward for ST: 1.87




In [53]:
#build utilitarian, selfish and greedy policies of each agent
Q_LT_Best = Q_LT_per_epoch[np.argmax(cumulative_reward_per_epoch)]
Q_MT_Best = Q_MT_per_epoch[np.argmax(cumulative_reward_per_epoch)]
Q_ST_Best = Q_ST_per_epoch[np.argmax(cumulative_reward_per_epoch)]

Q_LT_Immediate_Best = Q_LT_per_epoch[np.argmax(immediate_rewards_per_epoch[:, 0])]
Q_MT_Immediate_Best = Q_MT_per_epoch[np.argmax(immediate_rewards_per_epoch[:, 1])]
Q_ST_Immediate_Best = Q_ST_per_epoch[np.argmax(immediate_rewards_per_epoch[:, 2])]
                                                       
LT_Strategy = [LT_action_space[i] for i in Q_LT_Best[1:]]
MT_Strategy = [MT_action_space[i] for i in Q_MT_Best[1:]]
ST_Strategy = [ST_action_space[i] for i in Q_ST_Best[1:]]

LT_Greedy_Strategy = [LT_action_space[i] for i in Q_LT_Immediate_Best[1:]]
MT_Greedy_Strategy = [MT_action_space[i] for i in Q_MT_Immediate_Best[1:]]
ST_Greedy_Strategy = [ST_action_space[i] for i in Q_ST_Immediate_Best[1:]]

LT_Policy = [LT_action_space[i] for i in np.argmax(Q_LT, axis=1)]
MT_Policy = [MT_action_space[i] for i in np.argmax(Q_MT, axis=1)]
ST_Policy = [ST_action_space[i] for i in np.argmax(Q_ST, axis=1)]

print(f"LT's Strategy to achieve Highest Cumulative Reward: \n {LT_Strategy}")
print(f"LT's Strategy to achieve Highest Immediate Reward: \n {LT_Greedy_Strategy}")
print(f"Selfish Policy of LT, based on LT's Final Q-Table: \n {LT_Policy}")
print('\n')

print(f"MT's Strategy to achieve Highest Cumulative Reward: \n {MT_Strategy}")
print(f"MT's Strategy to achieve Highest Immediate Reward: \n {MT_Greedy_Strategy}")
print(f"Selfish Policy of MT, based on MT's Final Q-Table: \n {MT_Policy}")
print('\n')

print(f"ST's Strategy to achieve Highest Cumulative Reward: \n {ST_Strategy}")
print(f"ST's Strategy to achieve Highest Immediate Reward: \n {ST_Greedy_Strategy}")
print(f"Selfish Policy of ST, based on ST's Final Q-Table: \n {ST_Policy}")
print('\n')

LT's Strategy to achieve Highest Cumulative Reward: 
 [0.101, 0.101, 0.057, 0.079, 0.079, 0.079, 0.057, 0.079, 0.035, 0.057, 0.079, 0.079, 0.079, 0.079, 0.035, 0.035, 0.101, 0.101, 0.079, 0.079, 0.035, 0.035, 0.035, 0.035, 0.035, 0.035, 0.144, 0.079, 0.079, 0.035]
LT's Strategy to achieve Highest Immediate Reward: 
 [0.079, 0.079, 0.079, 0.079, 0.079, 0.101, 0.079, 0.079, 0.079, 0.079, 0.057, 0.079, 0.035, 0.079, 0.079, 0.101, 0.057, 0.079, 0.101, 0.079, 0.035, 0.035, 0.079, 0.035, 0.122, 0.035, 0.035, 0.079, 0.231, 0.035]
Selfish Policy of LT, based on LT's Final Q-Table: 
 [0.166, 0.122, 0.079, 0.079, 0.079, 0.079, 0.079, 0.101, 0.057, 0.079, 0.079, 0.057, 0.079, 0.122, 0.122, 0.079, 0.035, 0.101, 0.101, 0.057, 0.079, 0.035, 0.101, 0.101, 0.101, 0.144, 0.035, 0.101, 0.144, 0.231, 0.035]


MT's Strategy to achieve Highest Cumulative Reward: 
 [0.066, 0.066, 0.08, 0.073, 0.036, 0.015, 0.051, 0.036, 0.036, 0.015, 0.036, 0.036, 0.036, 0.029, 0.015, 0.015, 0.036, 0.015, 0.044, 0.015, 0.04

In [54]:
def Q_rowcount(Q_Table):
  subcount = 1
  for row in Q_Table:
    count = f"p{subcount}"
    print(count, row)
    subcount += 1

In [55]:
print(f"Final Q-Table of LT: \n {LT_action_space}")  #Q-Tables are printed...
print(Q_rowcount(Q_LT[:years]))
print('\n')

print(f"Final Q-Table of MT: \n {MT_action_space}")  #Q-Tables are printed...
print(Q_rowcount(Q_MT[:years]))
print('\n')

print(
    f"Final Q-Table of ST: \n {ST_action_space}"
)  #...Based on the "rewards_per_epoch" Table, the best Q-Tables are identified as policies
print(Q_rowcount(Q_ST[:years]))

Final Q-Table of LT: 
 [0.035 0.057 0.079 0.101 0.122 0.144 0.166 0.188 0.209 0.231]
p1 [2.124 1.931 2.209 2.128 0.    0.    2.373 0.    0.    0.   ]
p2 [2.499 2.581 2.58  1.495 2.631 0.    0.    0.    0.    0.   ]
p3 [2.125 2.014 2.551 0.98  0.    0.    0.    0.    0.    0.   ]
p4 [2.959 2.906 3.265 1.984 2.494 0.    0.    0.    0.    0.   ]
p5 [2.717 2.231 3.028 2.393 0.    0.    0.    0.    0.    0.   ]
p6 [2.349 2.584 2.672 0.    0.    0.    0.    0.    0.    0.   ]
p7 [2.334 2.549 2.667 1.684 0.    0.    0.    0.    0.    0.   ]
p8 [2.361 2.27  2.546 2.636 0.    0.    0.    0.    0.    0.   ]
p9 [3.364 3.589 3.493 0.    0.    0.    0.    0.    0.    0.   ]
p10 [3.383 3.268 3.645 0.    0.    0.    0.    0.    0.    0.   ]
p11 [3.467 3.377 3.521 3.223 0.    0.    0.    0.    0.    0.   ]
p12 [3.419 3.436 2.864 2.81  0.    0.    0.    0.    0.    0.   ]
p13 [3.65  2.791 3.812 2.922 1.045 0.    0.    0.    0.    0.   ]
p14 [3.334 3.535 3.628 0.    3.788 0.    0.    0.    0.    0.   ]


In [56]:
#Under utilary Policies
#initializing variables which are reset for every new epoch
global_state_of_co2_emission = 4.025  #current CO2 emissions are at 4.97 metric tons per capita world-wide. Source: World Bank
cumulative_reward_selfish = 0  #initializing cumulative reward, which is 0 to start with
alpha = 0.1  #initializing the learning rate of the Q-values
Q_LT = np.zeros((years+1, size_of_action_space))
Q_MT = np.zeros((years+1, size_of_action_space))
Q_ST = np.zeros((years+1, size_of_action_space))

#run period loop
for year in range(0,year):

    #actions based on Policies
    LT_action = LT_Strategy[year]
    MT_action = MT_Strategy[year]
    ST_action = ST_Strategy[year]

    #Immediate Reward functions
    LT_immediate_reward = -1 * LT_action * LT_reward_factor + cost_of_action * LT_action  #defining immediate reward function of LT per period
    MT_immediate_reward = -1 * LT_action * MT_reward_factor + cost_of_action * MT_action  #defining immediate reward function of LT per period
    ST_immediate_reward = -1 * ST_action * ST_reward_factor + cost_of_action * ST_action  #defining immediate reward function of ST per period

    #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
    Q_ST[year, abs(ST_action_space - ST_action).argmin()] = round(
            (1 - alpha) * Q_ST[year, Max_Q_ST] + alpha *
            (ST_immediate_reward + gamma * np.amax(Q_ST[year + 1, :])), 3)
        
        #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
    cumulative_reward_selfish += LT_immediate_reward + MT_immediate_reward + ST_immediate_reward - cost_of_action*((LT_action_space[np.argmax(Q_LT[year, :])]) - (MT_action_space[np.argmax(Q_MT[year, :])]) - (ST_action_space[np.argmax(Q_ST[year, :])]))
        
    global_state_of_co2_emission += ((LT_action*Population_LT[year]) + (MT_action*Population_MT[year]) + (ST_action*Population_ST[year]))/Population_array[year]

print('\n')
print(
    f"Cumulative Reward under Selfish Policies: {round(cumulative_reward_selfish, 2)}"
)
print(f"Global State after using Selfish Policies: {round(global_state_of_co2_emission, 2)}")

Selfish_Reward_Loss = round(
    100 *
    (cumulative_reward_selfish - np.max(cumulative_reward_per_epoch)) /
    np.max(cumulative_reward_per_epoch), 2)
CO2_Selfish = round(
    100 * ((global_state_of_co2_emission - np.min(global_state_per_epoch)) /
           np.min(global_state_per_epoch)), 2)

print('\n')
print(
    f"Percentage of Lower Cumulative Reward comparing Selfish Policies to Best Epoch: {Selfish_Reward_Loss}%"
)
print(
    f"Percentage of Higher Global State comparing Selfish Policies to Best Epoch: {CO2_Selfish}%"
)



Cumulative Reward under Selfish Policies: 160.5
Global State after using Selfish Policies: 4.67


Percentage of Lower Cumulative Reward comparing Selfish Policies to Best Epoch: 51.21%
Percentage of Higher Global State comparing Selfish Policies to Best Epoch: 5.25%


In [57]:
global_state_of_co2_emission

4.6650532470385055

In [58]:
#Under utilary Policies
#initializing variables which are reset for every new epoch
global_state = 4.025  #current CO2 emissions are at 4.97 metric tons per capita world-wide. Source: World Bank
cumulative_reward_Utilary = 0  #initializing cumulative reward, which is 0 to start with
alpha = 0.1  #initializing the learning rate of the Q-values
Q_LT = np.zeros((years+1, size_of_action_space))
Q_MT = np.zeros((years+1, size_of_action_space))
Q_ST = np.zeros((years+1, size_of_action_space))

#run period loop
for year in range(0,year):

    #actions based on Policies
    LT_action = LT_Policy[year]
    MT_action = MT_Policy[year]
    ST_action = ST_Policy[year]

    #Immediate Reward functions
    LT_immediate_reward = -1 * LT_action * LT_reward_factor + cost_of_action * LT_action  #defining immediate reward function of LT per period
    MT_immediate_reward = -1 * LT_action * MT_reward_factor + cost_of_action * MT_action  #defining immediate reward function of LT per period
    ST_immediate_reward = -1 * ST_action * ST_reward_factor + cost_of_action * ST_action  #defining immediate reward function of ST per period

    #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
    Q_ST[year, abs(ST_action_space - ST_action).argmin()] = round(
            (1 - alpha) * Q_ST[year, Max_Q_ST] + alpha *
            (ST_immediate_reward + gamma * np.amax(Q_ST[year + 1, :])), 3)
        
        #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
    cumulative_reward_Utilary += LT_immediate_reward + MT_immediate_reward + ST_immediate_reward - cost_of_action*((LT_action_space[np.argmax(Q_LT[year, :])]) - (MT_action_space[np.argmax(Q_MT[year, :])]) - (ST_action_space[np.argmax(Q_ST[year, :])]))
        
    global_state_of_co2_emission += ((LT_action*Population_LT[year]) + (MT_action*Population_MT[year]) + (ST_action*Population_ST[year]))/Population_array[year]

print('\n')
print(
    f"Cumulative Reward under Selfish Policies: {round(cumulative_reward_Utilary, 2)}"
)
print(f"Global State after using Selfish Policies: {round(global_state_of_co2_emission, 2)}")

Utilary_Reward_Loss = round(
    100 *
    (cumulative_reward_selfish - np.max(cumulative_reward_per_epoch)) /
    np.max(cumulative_reward_per_epoch), 2)
CO2_Utilary = round(
    100 * ((global_state - np.min(global_state_per_epoch)) /
           np.min(global_state_per_epoch)), 2)

print('\n')
print(
    f"Percentage of Lower Cumulative Reward comparing Utilary Policies to Best Epoch: {Selfish_Reward_Loss}%"
)
print(
    f"Percentage of Higher Global State comparing Utilary Policies to Best Epoch: {CO2_Selfish}%"
)



Cumulative Reward under Selfish Policies: 189.54
Global State after using Selfish Policies: 5.4


Percentage of Lower Cumulative Reward comparing Utilary Policies to Best Epoch: 51.21%
Percentage of Higher Global State comparing Utilary Policies to Best Epoch: 5.25%


In [59]:
#Under utilary Policies
#initializing variables which are reset for every new epoch
global_state = 4.025  #current CO2 emissions are at 4.97 metric tons per capita world-wide. Source: World Bank
cumulative_reward_Utilary = 0  #initializing cumulative reward, which is 0 to start with
alpha = 0.1  #initializing the learning rate of the Q-values
Q_LT = np.zeros((years+1, size_of_action_space))
Q_MT = np.zeros((years+1, size_of_action_space))
Q_ST = np.zeros((years+1, size_of_action_space))

#run period loop
for year in range(0,year):

    #actions based on Policies
    LT_action = LT_Greedy_Strategy[year]
    MT_action = MT_Greedy_Strategy[year]
    ST_action = ST_Greedy_Strategy[year]

    #Immediate Reward functions
    LT_immediate_reward = -1 * LT_action * LT_reward_factor + cost_of_action * LT_action  #defining immediate reward function of LT per period
    MT_immediate_reward = -1 * LT_action * MT_reward_factor + cost_of_action * MT_action  #defining immediate reward function of LT per period
    ST_immediate_reward = -1 * ST_action * ST_reward_factor + cost_of_action * ST_action  #defining immediate reward function of ST per period

    #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
    Q_ST[year, abs(ST_action_space - ST_action).argmin()] = round(
            (1 - alpha) * Q_ST[year, Max_Q_ST] + alpha *
            (ST_immediate_reward + gamma * np.amax(Q_ST[year + 1, :])), 3)
        
        #to be maximized: += immediate rewards of both agents + the negative of the actions that each agent takes, because the actions influence the global state, which shall be minimized
    cumulative_reward_Utilary += LT_immediate_reward + MT_immediate_reward + ST_immediate_reward - cost_of_action*((LT_action_space[np.argmax(Q_LT[year, :])]) - (MT_action_space[np.argmax(Q_MT[year, :])]) - (ST_action_space[np.argmax(Q_ST[year, :])]))
        
    global_state_of_co2_emission += ((LT_action*Population_LT[year]) + (MT_action*Population_MT[year]) + (ST_action*Population_ST[year]))/Population_array[year]

print('\n')
print(
    f"Cumulative Reward under Selfish Policies: {round(cumulative_reward_Utilary, 2)}"
)
print(f"Global State after using Selfish Policies: {round(global_state_of_co2_emission, 2)}")

Utilary_Reward_Loss = round(
    100 *
    (cumulative_reward_selfish - np.max(cumulative_reward_per_epoch)) /
    np.max(cumulative_reward_per_epoch), 2)
CO2_Utilary = round(
    100 * ((global_state - np.min(global_state_per_epoch)) /
           np.min(global_state_per_epoch)), 2)

print('\n')
print(
    f"Percentage of Lower Cumulative Reward comparing Utilary Policies to Best Epoch: {Selfish_Reward_Loss}%"
)
print(
    f"Percentage of Higher Global State comparing Utilary Policies to Best Epoch: {CO2_Selfish}%"
)



Cumulative Reward under Selfish Policies: 148.09
Global State after using Selfish Policies: 6.07


Percentage of Lower Cumulative Reward comparing Utilary Policies to Best Epoch: 51.21%
Percentage of Higher Global State comparing Utilary Policies to Best Epoch: 5.25%
